In [1]:
import os
import geopandas as gpd
import xarray as xr
from utils import *

import geopandas as gpd

In [2]:
folder = 'GRDC/guinea'
folder_out = 'GRDC/guinea/output'
if not os.path.exists(folder_out): os.makedirs(folder_out)

file_discharge = os.path.join(folder, 'GRDC-Monthly.nc')
file_stationbasins = os.path.join(folder, 'stationbasins.geojson')
file_subregions = os.path.join(folder, 'subregions.geojson')

In [3]:
# Make a map of data location
map_grdc_stationbasins_and_subregions(folder_out, file_stationbasins, file_subregions=None)

/opt/anaconda3/envs/HydroData/lib/python3.10/site-packages/pyogrio/raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


Map saved to map_stations_and_subregions.html


In [4]:
# Load data
data_location = gpd.read_file(file_stationbasins)
data_discharge = xr.open_dataset(file_discharge)

In [5]:
# Prepare export data
var_name = 'runoff_mean'

meta = data_discharge[["station_name", "geo_x", "geo_y"]].to_dataframe().reset_index()
area = data_discharge["area"].to_dataframe().reset_index()

# Convert to DataFrame
df = data_discharge[var_name].to_dataframe(name="Q").reset_index()

# Merge metadata into the main DataFrame
df = df.merge(meta, on="id")
df = df.merge(area, on="id")

# Add year and month columns
df["year"] = df["time"].dt.year
df["month"] = df["time"].dt.month

# Create a unique label for each station using name + coordinates (optional)
df["station_label"] = df["station_name"].str.strip() + " (" + df["geo_y"].round(2).astype(str) + ", " + df["geo_x"].round(2).astype(str) + ")"

# Pivot to wide format: year as index, MultiIndex (month, id) as columns
df_pivot = df.pivot(index="year", columns=["station_name", "month"], values="Q")
df_pivot.dropna(axis=1, how='all', inplace=True)
df_pivot.sort_index(ascending=True, axis=1, inplace=True)
df_pivot.to_csv(os.path.join(folder_out, 'GRDC_discharge_monthly-m3-s.csv'), index=True)

In [9]:
def filter_full_years(df):
    """
    Keep only (station, year) pairs with 12 valid months.
    Returns filtered DataFrame and number of dropped rows.
    """
    original_len = len(df)

    # Count valid months per station-year
    valid_counts = (
        df.groupby(['station_name', 'year'])['Q']
        .apply(lambda x: x.notna().sum())
        .reset_index(name='valid_months')
    )

    # Only keep those with all 12 months
    full_years = valid_counts[valid_counts['valid_months'] == 12]

    # Merge to filter original DataFrame
    df_filtered = df.merge(full_years[['station_name', 'year']], on=['station_name', 'year'])

    removed_rows = original_len - len(df_filtered)
    print(f"Removed {removed_rows} rows — kept {len(df_filtered)} only full (12-month) years.")

    return df_filtered

def discharge_to_runoff(df):
    """
    Convert annual discharge (m³/s) into annual runoff (mm/year) at the station level.

    The equation used is:

        runoff_mm = (Σ(Q_monthly_avg × 86400 × days_in_month)) / area_m2 × 1000

    Where:
        - Q is discharge in m³/s
        - 86400 is seconds per day
        - days_in_month accounts for monthly totals
        - area is the catchment area in m²
        - 1000 converts meters to millimeters

    Assumes:
        - Input DataFrame has one row per station/month
        - Years are complete (12 months per station)

    Returns:
        - DataFrame with columns: station_label, year, runoff_mm_year
    """
    df = df.copy()

    # Add number of days in each month
    df['days_in_month'] = pd.to_datetime(df['time']).dt.days_in_month

    # Convert area to m²
    df['area_m2'] = df['area'] * 1e6

    # Compute volume in m³ for each month
    df['volume_m3'] = df['Q'] * 86400 * df['days_in_month']

    # Sum monthly volumes per station-year
    runoff_by_year = (
        df.groupby(['station_name', 'year'])
        .apply(lambda x: x['volume_m3'].sum() / x['area_m2'].iloc[0] * 1000)  # m to mm
        .reset_index(name='runoff_mm_year')
    )

    return runoff_by_year

In [10]:
# Convert discharge data (m3-s) to runoff in (mm-year)
df_filtered = filter_full_years(df)
runoff_by_year = discharge_to_runoff(df_filtered)
# Save to CSV
runoff_by_year.round(0).pivot(index="year", columns="station_name", values="runoff_mm_year").to_csv(os.path.join(folder_out, 'GRDC_runoff_mm-year.csv'), index=True)

Removed 19332 rows — kept 3948 only full (12-month) years.


/var/folders/p9/3r4_fgzd72j7b469xxshgfnh0000gn/T/ipykernel_82814/3640971745.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x['volume_m3'].sum() / x['area_m2'].iloc[0] * 1000)  # m to mm


In [33]:
runoff_by_year

,station_label,year,runoff_mm_year
0,"BAC (8.06, -9.07)",1977,590.579331
1,"BAC (8.06, -9.07)",1978,811.859021
2,"BAC (8.06, -9.07)",1979,754.159085
3,"BAC (8.06, -9.07)",1980,590.713597
4,"BAC (8.06, -9.07)",1995,679.963798
...,...,...,...
324,"TINKISSO (11.25, -10.62)",1972,280.206021
325,"TINKISSO (11.25, -10.62)",1973,250.626850
326,"TINKISSO (11.25, -10.62)",1975,354.212934
327,"TINKISSO (11.25, -10.62)",1977,170.774893


In [34]:
def fill_missing_climatology(df, min_months=3):
    """
    Fill missing values using monthly climatology (station-wise).
    Returns a filled DataFrame and a mask of filled values.
    """
    df = df.copy()

    # Step 1: Filter out sparse years
    valid_years = (
        df.groupby(["station_label", "year"])["Q"]
        .apply(lambda x: x.notna().sum() >= min_months)
        .reset_index(name="keep")
    )

    # Merge to filter out sparse rows
    df = df.merge(valid_years[valid_years["keep"]], on=["station_label", "year"])
    df.drop(columns="keep", inplace=True)

    # Step 2: Build climatology
    climatology = (
        df.groupby(["station_label", "month"])["Q"]
        .mean()
        .rename("Q_clim")
        .reset_index()
    )

    df = df.merge(climatology, on=["station_label", "month"], how="left")

    # Step 3: Fill missing with climatology
    fill_mask = df["Q"].isna()
    df.loc[fill_mask, "Q"] = df.loc[fill_mask, "Q_clim"]

    df.drop(columns="Q_clim", inplace=True)

    return df_filled, filled_mask

def drop_sparse_years_and_interpolate(df, min_months=9):
    """
    Drops years with too much missing data and interpolates gaps per station.
    Assumes monthly data. Returns cleaned & interpolated DataFrame.
    """
    df_clean = df.copy()

    # Count non-NaN entries per year-station
    valid_counts = (
        df_clean.groupby(["station_label", "year"])["Q"]
        .apply(lambda x: x.notna().sum())
        .rename("valid_months")
        .reset_index()
    )

    # Keep only rows with enough months
    valid_years = valid_counts[valid_counts["valid_months"] >= min_months]
    df_clean = df_clean.merge(valid_years[["station_label", "year"]], on=["station_label", "year"])

    # Sort for interpolation
    df_clean = df_clean.sort_values(["station_label", "year", "month"])

    # Interpolate per station
    df_clean["Q"] = df_clean.groupby("station_label")["Q"].transform(lambda x: x.interpolate(method='linear', limit_direction='both'))

    return df_clean

def plot_all_fill_methods(df_original, df_clim, df_interp, output_dir):
    """
    Plots original, climatology-filled, and interpolated runoff data per station in one plot.

    Assumes all DataFrames have columns: ['year', 'month', 'station_label', 'Q'].
    """
    os.makedirs(output_dir, exist_ok=True)
    output_dir = os.path.join(output_dir, 'plots')
    os.makedirs(output_dir, exist_ok=True)

    # Combine 'year' and 'month' into datetime
    def add_datetime(df):
        return df.assign(
            date=pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str).str.zfill(2))
        )

    df_original = add_datetime(df_original)
    df_clim = add_datetime(df_clim)
    df_interp = add_datetime(df_interp)

    # Loop over each station
    stations = df_original['station_label'].unique()

    for station in stations:
        fig, ax = plt.subplots(figsize=(12, 4))

        # Subsets for current station
        df_o = df_original[df_original['station_label'] == station]
        df_c = df_clim[df_clim['station_label'] == station]
        df_i = df_interp[df_interp['station_label'] == station]

        # Plot all
        ax.plot(df_o['date'], df_o['Q'], label="Original", alpha=0.5, marker='o', linestyle='-', color='black')
        ax.plot(df_c['date'], df_c['Q'], label="Climatology Fill", linestyle='--', color='orange')
        ax.plot(df_i['date'], df_i['Q'], label="Interpolated", linestyle='-', color='blue')

        ax.set_title(f"Station: {station}")
        ax.set_ylabel("Discharge / Runoff (Q)")
        ax.set_xlabel("Time")
        ax.legend()
        ax.grid(True)

        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{station}.png"))
        plt.close()


In [35]:
df_filled, filled_mask = fill_missing_climatology(df, min_months=6)
df_interpolated = drop_sparse_years_and_interpolate(df, min_months=9)
plot_all_fill_methods(df, df_filled, df_interpolated, output_dir=folder)

NameError: name 'df_filled' is not defined